In [1]:
import os

import numpy as np
from astropy.coordinates import Angle

from pyuvdata import UVData
from pyuvdata.data import DATA_PATH

In [2]:
testfile = os.path.join(DATA_PATH, 'zen.2458661.23480.HH.uvh5')

uv_full = UVData()
uv_full.read(testfile)
uv_full.set_uvws_from_antenna_positions()

def check_cycle(drift_obj, phase_center_radec, use_antpos=True):
    assert drift_obj.phase_type == 'drift'
        
    cycle1 = drift_obj.copy()
    cycle2 = drift_obj.copy()

    cycle2.phase(ra=phase_center_radec[0],
                 dec=phase_center_radec[1],
                 use_ant_pos=use_antpos)
    cycle2.unphase_to_drift(use_ant_pos=use_antpos)

    # check that unphasing gets us back to what we started with
    uvw_diff = cycle2.uvw_array - drift_obj.uvw_array
    print("uvw diff, p/u vs none:", np.max(np.abs(uvw_diff)))

    vis_diff = cycle2.data_array - drift_obj.data_array
    print("vis diff, p/u vs none:", np.max(np.abs(vis_diff)))
    
    assert cycle2 == drift_obj

    # phase each the same way
    cycle1.phase(ra=phase_center_radec[0],
                 dec=phase_center_radec[1],
                 use_ant_pos=use_antpos)

    cycle2.phase(ra=phase_center_radec[0],
                 dec=phase_center_radec[1],
                 use_ant_pos=use_antpos)

    # check that phasing-unphasing-phasing is the same as phasing
    uvw_diff = cycle2.uvw_array - cycle1.uvw_array
    print("uvw diff, p/u/p vs p:", np.max(np.abs(uvw_diff)))

    vis_diff = cycle2.data_array - cycle1.data_array
    print("vis diff, p/u/p vs p:", np.max(np.abs(vis_diff)))

    # check that unphasing gets us back to what we started with
    cycle2.unphase_to_drift(use_ant_pos=use_antpos)

    uvw_diff = cycle2.uvw_array - drift_obj.uvw_array
    print("uvw diff, p/u/p/u vs none:", np.max(np.abs(uvw_diff)))

    vis_diff = cycle2.data_array - drift_obj.data_array
    print("vis diff, p/u/p/u vs none:", np.max(np.abs(vis_diff)))

    
#     assert cycle2 == cycle1
#     assert np.allclose(cycle2.data_array, cycle1.data_array)

In [3]:
check_cycle(uv_full, phase_center_radec = (Angle('0d').rad, Angle('-30d').rad))
check_cycle(uv_full, phase_center_radec = (Angle('0d').rad, Angle('0d').rad))

uvw diff, p/u vs none: 1.4210854715202004e-14
vis diff, p/u vs none: 4.1159031748919956e-11
uvw diff, p/u/p vs p: 0.0
vis diff, p/u/p vs p: 3.253907165344266e-11
uvw diff, p/u/p/u vs none: 1.4210854715202004e-14
vis diff, p/u/p/u vs none: 6.864124684985002e-11
uvw diff, p/u vs none: 1.4210854715202004e-14
vis diff, p/u vs none: 3.253907165344266e-11
uvw diff, p/u/p vs p: 0.0
vis diff, p/u/p vs p: 3.253907165344266e-11
uvw diff, p/u/p/u vs none: 1.4210854715202004e-14
vis diff, p/u/p/u vs none: 6.507814330688532e-11


In [4]:
check_cycle(uv_full, phase_center_radec = (Angle('0d').rad, Angle('-30d').rad), use_antpos=False)
check_cycle(uv_full, phase_center_radec = (Angle('0d').rad, Angle('0d').rad), use_antpos=False)

uvw diff, p/u vs none: 9.913153802454389e-05
vis diff, p/u vs none: 3.637978807091713e-11
uvw diff, p/u/p vs p: 3.0517578125e-05
vis diff, p/u/p vs p: 12.891395034519087
uvw diff, p/u/p/u vs none: 9.524571402241833e-05
vis diff, p/u/p/u vs none: 2.328306436606459e-10
uvw diff, p/u vs none: 9.913153802454389e-05
vis diff, p/u vs none: 3.253907165344266e-11
uvw diff, p/u/p vs p: 3.0517578125e-05
vis diff, p/u/p vs p: 2.6647776794889158
uvw diff, p/u/p/u vs none: 9.524571402241833e-05
vis diff, p/u/p/u vs none: 2.3283064365429315e-10


In [5]:
testfile = os.path.join(DATA_PATH, 'zen.2458661.23480.HH.uvh5')
uv_full = UVData()
uv_full.read_uvh5(testfile)
print(uv_full.phase_type)

initial_uvws = uv_full.uvw_array

uv_full.set_uvws_from_antenna_positions()

uvw_diff = initial_uvws - uv_full.uvw_array
print('max uvw difference:', np.max(np.abs(uvw_diff)))

print('max u difference:', np.max(np.abs(uvw_diff[:, 0])))
print('max v difference:', np.max(np.abs(uvw_diff[:, 1])))
print('max w difference:', np.max(np.abs(uvw_diff[:, 2])))


drift
max uvw difference: 73.13580521406377
max u difference: 73.13580521406377
max v difference: 25.23781680567602
max w difference: 0.2998355882363697


In [6]:
uv_full.reorder_blts(order="time", minor_order="baseline")
times = np.unique(uv_full.time_array)
time_set_1, time_set_2 = times[::2], times[1::2]

uv_early = uv_full.select(times=time_set_1, inplace=False)
uv_late = uv_full.select(times=time_set_2, inplace=False)

uv_early_drift = uv_early.copy()
uv_late_drift = uv_late.copy()
uv_drift = uv_full.copy()


In [7]:
phase_center_radec = (Angle('0d').rad, Angle('-30d').rad)

# phase each half to different spots
uv_early.phase(ra=0, dec=0, use_ant_pos=True)

uv_late.phase(ra=phase_center_radec[0],
               dec=phase_center_radec[1],
               use_ant_pos=True)

# phase original to phase_center_radec
uv_full.phase(ra=phase_center_radec[0],
             dec=phase_center_radec[1],
             use_ant_pos=True)


In [9]:
uv_out = uv_early.fast_concat(uv_late, axis='blt', phase_center_radec=phase_center_radec)
uv_out.reorder_blts(order='time', minor_order='baseline')

Phasing this UVData object to phase_center_radec


In [10]:
assert uv_out.phase_type == "phased"
uv_out.history = uv_full.history
assert (uv_out.phase_center_ra, uv_out.phase_center_dec) == phase_center_radec
assert uv_out == uv_full
